In [2]:
import json
import yaml
import uuid
import os
from pathlib import Path

In [3]:
import pandas as pd

In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload 2

In [11]:
import wombat.pecgs as pecgs

###### make pecgs pipeline cwl

In [15]:
cwl_dir = '/home/estorrs/pecgs-pipeline/cwl/pecgs_workflows'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [41]:
template = {
    'sample': 'a_string',
    'cpu': 'a_int',
    'tumor_wxs_fq_1': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'tumor_wxs_fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'normal_wxs_fq_1': {
        'class': 'File', 
        'path': 'a/file/path'
    },
    'normal_wxs_fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'tumor_rna_fq_1': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'tumor_rna_fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    
    ## align dna-seq
    'reference': {
        'class': 'File',
        'path': 'path/to/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.dict'}
        ]
    },
    'known_sites': {
        'class': 'File',
        'path': 'path/to/vcf.gz',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/vcf.gz.tbi'}
        ]
    },
    'wxs_normal_flowcell': 'a_string',
    'wxs_normal_lane': 'a_string',
    'wxs_normal_index_sequencer': 'a_string',
    'wxs_normal_library_preparation': 'a_string',
    'wxs_normal_platform': 'a_string',
    'wxs_tumor_flowcell': 'a_string',
    'wxs_tumor_lane': 'a_string',
    'wxs_tumor_index_sequencer': 'a_string',
    'wxs_tumor_library_preparation': 'a_string',
    'wxs_tumor_platform': 'a_string',
    
    # fusion
    'genome_lib_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'genome_db': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'bwts': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'integrate_executable': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_fasta': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_annotations': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'filter_database': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'fusion_annotator_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    
    # CNV
    'reference_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'target_interval_list': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'common_biallelic': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'protein_coding_gene': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'pool_of_normals': {
        'class': 'File',
        'path': 'a/file/path'
    },
    
    # msisensor
    'microsatellite': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'minimal_homopolymer_size': 'a_int',
    'minimal_microsatellite_size': 'a_int',
    
    # TinDaisy
    'rescue_clinvar': False,
    'rescue_cosmic': False,
    'vep_cache_version': 'a_string',
    'vep_cache_gz': {'class': 'File', 'path': 'a/file/path'},
    'clinvar_annotation': {'class': 'File', 'path': 'a/file/path'},
    'classification_config': {'class': 'File', 'path': 'a/file/path'},
    'af_config': {'class': 'File', 'path': 'a/file/path'},
    'call_regions': {'class': 'File', 'path': 'a/file/path'},
    'canonical_BED': {'class': 'File', 'path': 'a/file/path'},
    'normal_barcode': 'a_string',
    'tumor_barcode': 'a_string',
    'tindaisy_chrlist': {'class': 'File', 'path': 'a/file/path'},
    'strelka_config': {'class': 'File', 'path': 'a/file/path'},
    'centromere_bed': {'class': 'File', 'path': 'a/file/path'},
    'assembly': 'a_string',
    'varscan_config': {'class': 'File', 'path': 'a/file/path'},
    'pindel_config': {'class': 'File', 'path': 'a/file/path'},
    
    # TinJasmine
    'sample_barcode': 'a_string',
    'centromere': {'class': 'File', 'path': 'a/file/path'},
    'tinjasmine_chrlist': {'class': 'File', 'path': 'a/file/path'},
    'Canonical_BED': {'class': 'File', 'path': 'a/file/path'},
    'ROI_BED': {'class': 'File', 'path': 'a/file/path'},
    'varscan_filter_config': {'class': 'File', 'path': 'a/file/path'},
    'pindel_filter_config': {'class': 'File', 'path': 'a/file/path'},
    'pindel_config_template': {'class': 'File', 'path': 'a/file/path'},
    'gatk_filter_config': {'class': 'File', 'path': 'a/file/path'}, 

}

In [42]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.pecgs_TN_wxs_fq_T_rna_fq.yaml'), 'w'))

In [43]:
cwl = {
    'cwlVersion': 'v1.0',
    'class': 'Workflow',
    'id': 'pecgs_TN_wxs_fq_T_rna_fq',
    'label': 'pecgs_TN_wxs_fq_T_rna_fq',
    'inputs': {
        'sample': 'string',
        'cpu': {
            'type': 'int?',
            'default': 40
        },
        'tumor_wxs_fq_1': {
            'type': 'File'
        },
        'tumor_wxs_fq_2': {
            'type': 'File'
        },
        'normal_wxs_fq_1': {
            'type': 'File'
        },
        'normal_wxs_fq_2': {
            'type': 'File'
        },
        'tumor_rna_fq_1': {
            'type': 'File'
        },
        'tumor_rna_fq_2': {
            'type': 'File'
        },
        
        # align dna-seq
        'tumor_sample': {
            'type': 'string?',
            'default': '$(inputs.sample).WXS.T'
        },
        'normal_sample': {
            'type': 'string?',
            'default': '$(inputs.sample).WXS.N'
        },
        'known_sites': {
            'type': 'File',
           'secondaryFiles': ['.tbi']
        },
        'reference': {
           'type': 'File',
           'secondaryFiles': [
               '.amb',
               '.ann',
               '.bwt',
               '.fai',
               '.pac',
               '.sa',
               '^.dict'
           ]
        },
        'wxs_normal_flowcell': {
            'type': 'string?',
            'default': 'flowcellABCDE'
        },
        'wxs_normal_lane': {
            'type': 'string?',
            'default': '1'
        },
        'wxs_normal_index_sequencer': {
            'type': 'string?',
            'default': 'sequencerABCDE'
        },
        'wxs_normal_library_preparation': {
            'type': 'string?',
            'default': 'lib1'
        },
        'wxs_normal_platform': {
            'type': 'string?',
            'default': 'ILLUMINA'
        },
        'wxs_tumor_flowcell': {
            'type': 'string?',
            'default': 'flowcellABCDE'
        },
        'wxs_tumor_lane': {
            'type': 'string?',
            'default': '1'
        },
        'wxs_tumor_index_sequencer': {
            'type': 'string?',
            'default': 'sequencerABCDE'
        },
        'wxs_tumor_library_preparation': {
            'type': 'string?',
            'default': 'lib1'
        },
        'wxs_tumor_platform': {
            'type': 'string?',
            'default': 'ILLUMINA'
        },
        
        # fusion
        'genome_lib_dir': {
            'type': 'Directory',
        },
        'genome_db': {
            'type': 'Directory',
        },
        'bwts': {
            'type': 'Directory',
        },
        'integrate_executable': {
            'type': 'File',
        },
        'integrate_fasta': {
            'type': 'File',
        },
        'integrate_annotations': {
            'type': 'File',
        },
        'filter_database': {
            'type': 'Directory',
        },
        'fusion_annotator_dir': {
            'type': 'Directory',
        },
        
        # CNV
        'reference_dir': {
            'type': 'Directory',
        },
        'target_interval_list': {
            'type': 'File',
        },
        'common_biallelic': {
            'type': 'File',
        },
        'protein_coding_gene': {
            'type': 'File',
        },
        'pool_of_normals': {
            'type': 'File',
        },
        
        # msisensor
        'microsatellite': {
            'type': 'File',
        },
        'minimal_homopolymer_size': {
            'type': 'int?',
            'default': 1
        },
        'minimal_microsatellite_size': {
            'type': 'int?',
            'default': 1
        },
        
        # TinDaisy
        'rescue_clinvar': {
            'type': 'boolean?',
            'default': False,
        },
        'rescue_cosmic': {
            'type': 'boolean?',
            'default': False,
        },
        'vep_cache_version': {
            'type': 'string'
        },
        'vep_cache_gz': {
            'type': 'File'
        },
        'clinvar_annotation': {
            'type': 'File'
        },
        'classification_config': {
            'type': 'File'
        },
        'af_config': {
            'type': 'File'
        },
        'call_regions': {
            'type': 'File'
        },
        'canonical_BED': {
            'type': 'File'
        },
        'normal_barcode': {
            'type': 'string?',
            'default': '$(inputs.sample).N',
        },
        'tumor_barcode': {
            'type': 'string?',
            'default': '$(inputs.sample).T',
        },
        'tindaisy_chrlist': {
            'type': 'File'
        },
        'strelka_config': {
            'type': 'File'
        },
        'centromere_bed': {
            'type': 'File'
        },
        'assembly': {
            'type': 'string'
        },
        'varscan_config': {
            'type': 'File'
        },
        'pindel_config': {
            'type': 'File'
        },
        
        # TinJasmine
        'sample_barcode': {
            'type': 'string?',
            'default': '$(inputs.sample)'
        },
        'centromere': {
            'type': 'File'
        },
        'tinjasmine_chrlist': {
            'type': 'File'
        },
        'Canonical_BED': {
            'type': 'File'
        },
        'ROI_BED': {
            'type': 'File'
        },
        'varscan_filter_config': {
            'type': 'File'
        },
        'pindel_filter_config': {
            'type': 'File'
        },
        'pindel_config_template': {
            'type': 'File'
        },
        'gatk_filter_config': {
            'type': 'File'
        }
    },
    'outputs': {
        'tumor_wxs_output_bam': {
            'type': 'File',
            'outputSource': 'align_tumor_wxs/output_bam',
            'secondaryFiles': ['^.bai']
        },
        'normal_wxs_output_bam': {
            'type': 'File',
            'outputSource': 'align_normal_wxs/output_bam',
            'secondaryFiles': ['^.bai']
        },
        'filtered_fusions': {
            'type': 'File',
            'outputSource': 'run_fusion/filtered_fusions'
        },
        'total_fusions': {
            'type': 'File',
            'outputSource': 'run_fusion/total_fusions'
        },
        'gene_level_cnv': {
            'type': 'File',
            'outputSource': 'run_cnv/gene_level_cnv'
        },
        'msisensor_output_summary': {
            'type': 'File',
            'outputSource': 'run_msisensor/output_summary'
        },
        'msisensor_output_dis': {
            'type': 'File',
            'outputSource': 'run_msisensor/output_dis'
        },
        'msisensor_output_germline': {
            'type': 'File',
            'outputSource': 'run_msisensor/output_germline'
        },
        'msisensor_output_somatic': {
            'type': 'File',
            'outputSource': 'run_msisensor/output_somatic'
        },
        'tindaisy_output_maf_clean': {
            'type': 'File',
            'outputSource': 'run_tindaisy/output_maf_clean'
        },
        'tindaisy_output_vcf_clean': {
            'type': 'File',
            'outputSource': 'run_tindaisy/output_vcf_clean'
        },
        'tindaisy_output_vcf_all': {
            'type': 'File',
            'outputSource': 'run_tindaisy/output_vcf_all'
        },
        'tinjasmine_output_maf_clean': {
            'type': 'File',
            'outputSource': 'run_tinjasmine/clean_MAF'
        },
        'tinjasmine_output_vcf_clean': {
            'type': 'File',
            'outputSource': 'run_tinjasmine/clean_VCF'
        },
        'tinjasmine_output_vcf_all': {
            'type': 'File',
            'outputSource': 'run_tinjasmine/allCall_VCF'
        },
    },
    'steps': {
        'align_tumor_wxs': {
            'run': '../../submodules/align-dnaseq/cwl/align_dnaseq.cwl',
            'in': {
                'sample': 'tumor_sample',
                'cpu': 'cpu',
                'fq_1': 'tumor_wxs_fq_1',
                'fq_2': 'tumor_wxs_fq_1',
                'known_sites': 'known_sites',
                'reference': 'reference',
                'flowcell': 'flowcell',
                'lane': 'lane',
                'index_sequencer': 'index_sequencer',
                'library_preparation': 'library_preparation',
                'platform': 'platform',  
            },
            'out': [
                {'id': 'output_bam'}
            ]
        },
        'align_normal_wxs': {
            'run': '../../submodules/align-dnaseq/cwl/align_dnaseq.cwl',
            'in': {
                'sample': 'normal_sample',
                'cpu': 'cpu',
                'fq_1': 'normal_wxs_fq_1',
                'fq_2': 'normal_wxs_fq_1',
                'known_sites': 'known_sites',
                'reference': 'reference',
                'flowcell': 'flowcell',
                'lane': 'lane',
                'index_sequencer': 'index_sequencer',
                'library_preparation': 'library_preparation',
                'platform': 'platform',  
            },
            'out': [
                {'id': 'output_bam'}
            ]
        },
        'run_fusion': {
            'run': '../../submodules/pecgs-fusion/cwl/fusion.cwl',
            'in': {
                'sample': 'sample',
                'cpu': 'cpu',
                'fq1': 'tumor_rna_fq_1',
                'fq2': 'tumor_rna_fq_2',
                'filter_database': 'filter_database',
                'bwts': 'bwts',
                'fusion_annotator_dir': 'fusion_annotator_dir',
                'genome_db': 'genome_db',
                'genome_lib_dir': 'genome_lib_dir',
                'integrate_annotations': 'integrate_annotations',
                'integrate_executable': 'integrate_executable',
                'integrate_fasta': 'integrate_fasta'
            },
            'out': [
                {'id': 'filtered_fusions'},
                {'id': 'total_fusions'}
            ]
        },
        'run_cnv': {
            'run': '../../submodules/pecgs-cnv/cwl/cnv_workflow.cwl',
            'in': {
                'sample': 'sample',
                'cpu': 'cpu',
                'tumor_bam': 'align_tumor_wxs/output_bam',
                'normal_bam': 'align_normal_wxs/output_bam',
                'reference_dir': 'reference_dir',
                'target_interval_list': 'target_interval_list',
                'common_biallelic': 'common_biallelic',
                'protein_coding_gene': 'protein_coding_gene',
                'pool_of_normals': 'pool_of_normals',
            },
            'out': [
                {'id': 'gene_level_cnv'}
            ]
        },
        'run_msisensor': {
            'run': '../msisensor/msisensor_workflow.cwl',
            'in': {
                'threads': 'cpu',
                'tumor_bam': 'align_tumor_wxs/output_bam',
                'normal_bam': 'align_normal_wxs/output_bam',
                'microsatellite': 'microsatellite',
                'minimal_homopolymer_size': 'minimal_homopolymer_size',
                'minimal_microsatellite_size': 'minimal_microsatellite_size',
            },
            'out': [
                {'id': 'output_summary'},
                {'id': 'output_dis'},
                {'id': 'output_germline'},
                {'id': 'output_somatic'},
            ]
        },
        'run_tindaisy': {
            'run': '../../submodules/TinDaisy/cwl/workflows/tindaisy2.cwl',
            'in': {
                'tumor_bam': 'align_tumor_wxs/output_bam',
                'normal_bam': 'align_normal_wxs/output_bam',
                'reference_fasta': 'reference',
                'pindel_config': 'pindel_config',
                'varscan_config': 'varscan_config',
                'assembly': 'assembly',
                'centromere_bed': 'centromere_bed',
                'strelka_config': 'strelka_config',
                'chrlist': 'tindaisy_chrlist',
                'tumor_barcode': 'tumor_barcode',
                'normal_barcode': 'normal_barcode',
                'canonical_BED': 'canonical_BED',
                'call_regions': 'call_regions',
                'af_config': 'af_config',
                'classification_config': 'classification_config',
                'clinvar_annotation': 'clinvar_annotation',
                'vep_cache_gz': 'vep_cache_gz',
                'vep_cache_version': 'vep_cache_version',
                'rescue_cosmic': 'rescue_cosmic',
                'rescue_clinvar': 'rescue_clinvar',
            },
            'out': [
                {'id': 'output_maf_clean'},
                {'id': 'output_vcf_clean'},
                {'id': 'output_vcf_all'},
            ]
        },
        'run_tinjasmine': {
            'run': '../../submodules/TinJasmine/cwl/TinJasmine.cwl',
            'in': {
                'sample_barcode': 'normal_barcode',
                'bam': 'align_normal_wxs/output_bam',
                'reference': 'reference',
                'gatk_filter_config': 'gatk_filter_config',
                'pindel_config_template': 'pindel_config_template',
                'pindel_filter_config': 'pindel_filter_config',
                'varscan_filter_config': 'varscan_filter_config',
                'ROI_BED': 'ROI_BED',
                'vep_cache_gz': 'vep_cache_gz',
                'vep_cache_version': 'vep_cache_version',
                'assembly': 'assembly',
                'Canonical_BED': 'Canonical_BED',
                'chrlist': 'tinjasmine_chrlist',
                'centromere': 'centromere',
            },
            'out': [
                {'id': 'clean_VCF'},
                {'id': 'allCall_VCF'},
                {'id': 'clean_MAF'},
            ]
        },
        
    },
   'requirements': [
       {'class': 'InlineJavascriptRequirement'},
       {'class': 'MultipleInputFeatureRequirement'},
       {'class': 'ScatterFeatureRequirement'},
       {'class': 'StepInputExpressionRequirement'},
       {'class': 'SubworkflowFeatureRequirement'}
   ]
}

In [44]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'pecgs_TN_wxs_fq_T_rna_fq.cwl'), 'w'))

###### test run

In [6]:
run_list = pd.read_csv('../tests/data/pecgs_pipeline/run_list.txt', sep='\t', index_col=0)
run_list

,rna-seq_tumor_R1.filepath,rna-seq_tumor_R1.uuid,rna-seq_tumor_R2.filepath,rna-seq_tumor_R2.uuid,wxs_normal_R1.filepath,wxs_normal_R1.uuid,wxs_normal_R2.filepath,wxs_normal_R2.uuid,wxs_tumor_R1.filepath,wxs_tumor_R1.uuid,wxs_tumor_R2.filepath,wxs_tumor_R2.uuid,sample_id,run_uuid
run_id,,,,,,,,,,,,,,
HT191P1-S1H1A3Y3_23402d1f-7ea6-48d0-8553-02d5f4163aed,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,ab2eb486-8926-4329-b14c-81e9605edcc4,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,530103e2-ecc2-4400-b89b-329fb16631b0,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,ee5aa566-1492-49bd-a44f-70a28a9bd6e1,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,08739a15-9bb4-4ab1-82e4-67ac523d65e4,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,53ccd273-07b5-416f-a68a-bd4953424612,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,6fb1f30a-a19a-4702-9707-7d072d13222e,HT191P1-S1H1A3Y3,23402d1f-7ea6-48d0-8553-02d5f4163aed


In [7]:
run_map = run_list.transpose().to_dict()
run_map = {k:{c.replace('.filepath', ''):val for c, val in v.items() if 'filepath' in c}
          for k, v in run_map.items()}
run_map

{'HT191P1-S1H1A3Y3_23402d1f-7ea6-48d0-8553-02d5f4163aed': {'rna-seq_tumor_R1': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG.HFWJGDSXY_AAGGTGTTAG-TAAGCGCGTG_L001_R1.fastq.gz',
  'rna-seq_tumor_R2': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG.HFWJGDSXY_AAGGTGTTAG-TAAGCGCGTG_L001_R1.fastq.gz',
  'wxs_normal_R1': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R1_001.fastq.gz',
  'wxs_normal_R2': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R2_001.fastq.gz',
  'wxs_tumor_R1': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CCAGTAGCGT-ATGTATTGGC_S53_L002_R1_001.fastq.gz',
  'wxs_tu

In [8]:
sequencing_info = yaml.safe_load(open('../tests/data/pecgs_pipeline/sequencing_info.yaml'))
sequencing_info

{'HT191P1-S1H1A3Y3_23402d1f-7ea6-48d0-8553-02d5f4163aed': {'normal': {'flowcell': 'HH7KNDSXY',
   'index_sequencer': 'CATTATCGCT-CTTGAAGGTT',
   'lane': '1',
   'library_preparation': 'TWCE-HT191P1-JM1D1_1-lib1',
   'platform': 'ILLUMINA'},
  'tumor': {'flowcell': 'HFMFWDSXY',
   'index_sequencer': 'CCAGTAGCGT-ATGTATTGGC',
   'lane': '2',
   'library_preparation': 'TWCE-HT191P1-S1H1A3Y3D1_1-lib1',
   'platform': 'ILLUMINA'}}}

In [9]:
run_dir = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs'
proxy_run_dir = '../tests/data/pecgs_pipeline/run_dir'
tool_root = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

In [20]:
start_cmds, server_cmds, job_cmds = pecgs.from_run_list_TN_wxs_fq_T_rna_fq(
    run_map, run_dir, tool_root, sequencing_info_map=sequencing_info, proxy_run_dir=proxy_run_dir)

In [21]:
start_cmds

['source /opt/ibm/lsfsuite/lsf/conf/lsf.conf',
 'export LSF_DOCKER_NETWORK=host',
 'export LSF_DOCKER_VOLUMES="/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"',
 'bgadd -L 50 /estorrs/default_pecgs_pipeline',
 "bsub -n 1 -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/default_pecgs_pipeline -J b70f2017-e818-43e2-bfa8-009a831b08a6 -Is '/bin/bash'"]

In [22]:
server_cmds

'/usr/bin/java -Dconfig.file=/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs/inputs/server-cromwell-config.compute1.dat -jar /usr/local/cromwell/cromwell-47.jar server'

In [23]:
job_cmds

['mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs/logs',
 'mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs/runs/HT191P1-S1H1A3Y3_23402d1f-7ea6-48d0-8553-02d5f4163aed/cromwell-workdir/logs',
 'source /opt/ibm/lsfsuite/lsf/conf/lsf.conf',
 'export LSF_DOCKER_NETWORK=host',
 'export LSF_DOCKER_VOLUMES="/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"',
 "bsub -n 1 -q dinglab -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/default_pecgs_pipeline -J cromwell_launch_HT191P1-S1H1A3Y3_23402d1f-7ea6-48d0-8553-02d5f4163aed -oo /scratch1/fs1/dinglab/est